In [8]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

##################################################
# Load  
##################################################
input_col = ['msno','w','days_since_the_last_expiration',
'days_since_the_last_subscription','days_since_the_last_expiration-cumsum',
'days_since_the_last_expiration_ratio','days_since_the_last_subscription_ratio',
'days_since_the_first_subscription']
input_col = ['msno','w','is_subscribe_early','do_change_payment_method','do_spend_more_money', 
    'do_extend_payment_days', 'do_paid_more']

In [9]:
T = 0
folder = 'trainW-'+str(T)
membership_loyalty = utils.read_multiple_csv('../../feature/{}/days_since_the_last_transactions'.format(folder), input_col) 
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','w']] # we do not need is_churn
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(membership_loyalty)


100%|██████████| 4/4 [00:27<00:00,  6.82s/it]


reduce memory


100%|██████████| 7/7 [00:00<00:00, 1602.89it/s]


In [7]:
membership_loyalty.columns

Index(['msno', 'w', 'days_since_the_last_expiration',
       'days_since_the_last_subscription', 'is_subscribe_early',
       'order_number_rev', 'days_since_the_last_expiration-cumsum',
       'days_since_the_last_expiration_ratio',
       'days_since_the_last_subscription_ratio',
       'days_since_the_first_subscription', 'do_change_payment_method',
       'do_spend_more_money', 'do_extend_payment_days', 'do_paid_more'],
      dtype='object')

In [ ]:
input_col = ['msno','w','is_subscribe_early','do_change_payment_method','do_spend_more_money', 
    'do_extend_payment_days', 'do_paid_more']

In [19]:
df = pd.merge(train, 
    membership_loyalty, 
    on=['msno','w'], 
    how='left')


In [20]:
df.head(n= 100)

,msno,w,is_subscribe_early,do_change_payment_method,do_spend_more_money,do_extend_payment_days,do_paid_more
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
3,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,1.0,0.0,0.0,0.0,0.0
4,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
5,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,1.0,0.0,0.0,0.0,0.0
6,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,1.0,0.0,0.0,0.0,0.0
7,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
8,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0
9,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,0.0,0.0,0.0,0.0,0.0


In [12]:
def near(x, keep = 5):
    return x.tail(keep)
def make_order_number(x):
    x['order_number'] = [i+1 for i in range(x.shape[0])]
    return x


In [36]:
df[df.msno == '++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=']

,msno,w,is_subscribe_early,do_change_payment_method,do_spend_more_money,do_extend_payment_days,do_paid_more
12528954,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0,0.0,0.0,0.0,0.0,0.0
12528955,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0,0.0,0.0,0.0,0.0,0.0
12528956,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0,1.0,1.0,-149.0,-23.0,0.0
12528957,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0,0.0,1.0,149.0,23.0,0.0
12528958,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0,0.0,0.0,328.0,90.0,0.0


In [15]:
df = df.head(n = 1000)

In [33]:
	tbl = df.groupby('msno').do_extend_payment_days.sum().to_frame()
	tbl.columns = ['do_extend_payment_days_count']
	tbl['do_extend_payment_days_ratio'] = df.groupby('msno').do_extend_payment_days.mean()
	tbl.reset_index(inplace = True)


In [39]:
tbl = df.groupby('msno').do_spend_more_money.mean().to_frame()
tbl.columns = ['do_spend_more_money-mean']
tbl['do_spend_more_money-min'] = df.groupby('msno').do_spend_more_money.min()
tbl['do_spend_more_money-max'] = df.groupby('msno').do_spend_more_money.max()
tbl['do_spend_more_money-median'] = df.groupby('msno').do_spend_more_money.median()
tbl['do_spend_more_money-std'] = df.groupby('msno').do_spend_more_money.std()
tbl.reset_index(inplace = True)

In [41]:
tbl[tbl['do_spend_more_money-median'] != 0]

,msno,do_spend_more_money-mean,do_spend_more_money-min,do_spend_more_money-max,do_spend_more_money-median,do_spend_more_money-std
444,+02xbGYfe7Reo33j751VD2WpZwJYb0156Jgq/O4XZYI=,15.50,0.0,31.0,15.5,21.920310
672,+15tabgS1TqSjJ6bmWK5lYfFc+Mqd7Q5MvcL3nup+ug=,74.50,0.0,149.0,74.5,105.358910
800,+1joHQ8EPnzLJX3VMAQjEST6mA1iHe7DHYE4cx0u+rE=,-357.00,-1608.0,894.0,-357.0,1769.181167
885,+23TJD19Y0skStAvfWMyGR4tRSaZJ+V5a4lZ+idFam4=,90.00,0.0,180.0,90.0,127.279221
1001,+2ZtlB7ndAxAinxEeKE58U2vynBwND9MntU4DKVR0rk=,-40.00,-80.0,0.0,-40.0,56.568542
1042,+2oK/qWmYvAnfNZsVV5pdsJ9n6d/LZn6CdwiJajGZas=,-1608.00,-1608.0,-1608.0,-1608.0,NaN
1690,+5gJ1LrG0CAJ4vdkxwkSqMprOGuDdhO7exXxTJOjaYU=,NaN,NaN,NaN,NaN,NaN
1884,+6bh6y64hUlz9zESpXTCYo6VfOvgh50pwn0HBnW8wBA=,-165.50,-328.0,-3.0,-165.5,229.809704
2253,+8HOhO0neZY8AOCN8pd4w7D4lxddmCK1AMDzV4QIJt0=,-804.00,-1608.0,0.0,-804.0,1137.027704
2669,+AD+b9knTeCZePOKvDXz382f1QTfIcKlHKIMY3KyAak=,-80.00,-80.0,-80.0,-80.0,NaN
